In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import glob
# from rasterstats import zonal_stats

In [3]:
def city_stats(data):
    # Data I need
    # 1. Majority cluster of each zone
    # 2. Majority cluster of each city (all zones)
    # 3. Percentages of each zone
    # 4. Percentages of each city
    histos = data.iloc[:, 2:]
    majority = histos.idxmax(axis=1)
    maj_count = majority.value_counts()

    perc_list = pd.DataFrame()
    for index, row in histos.iterrows():
        percentage = (row / row.sum()) * 100
        perc_list = pd.concat([perc_list, percentage], axis=1)
    perc_list = perc_list.T

    city_sum = perc_list.sum()
    city_perc = (city_sum / city_sum.sum()) * 100

    return majority, maj_count, perc_list, city_perc

In [4]:
slum_zones = gpd.read_file("data/zonal_statistics/nairobi_36-86_sdi_histogram.dbf")
print(slum_zones)

     fid   Id Country     City               Settlement  Date_updat   kyc_pop  \
0    1.0   85   Kenya  Nairobi               Commercial  25.05.2016    5782.0   
1    2.0  108   Kenya  Nairobi         Kaberera Village  25.05.2016    2465.0   
2    3.0   86   Kenya  Nairobi                 Crescent  25.05.2016   31003.0   
3    4.0  110   Kenya  Nairobi               Kambi Moto  23.02.2018   36990.0   
4    5.0  137   Kenya  Nairobi     Mandazi Road Village  10.05.2016    3546.0   
5    6.0  118   Kenya  Nairobi    Kenya Wine Settlement  10.05.2016    1560.0   
6    7.0   89   Kenya  Nairobi                  Diamond  31.05.2016     768.0   
7    8.0   95   Kenya  Nairobi                  Gateway  30.05.2016   10996.0   
8    9.0  129   Kenya  Nairobi                   Kosovo  30.05.2016    7384.0   
9   10.0   79   Kenya  Nairobi                     Bins  15.06.2016   13419.0   
10  11.0   93   Kenya  Nairobi        Feed The Children  30.05.2016    6090.0   
11  12.0   96   Kenya  Nairo

In [5]:
cols = ['Settlement', 'Population', 'kyc_pop', 'HISTO_0', 'HISTO_1', 'HISTO_2', 'HISTO_3', 'HISTO_4', 'HISTO_5']
city_data_list = []
city_dict_list = []

for filepath in glob.iglob("data/zonal_statistics/*.dbf"):
    shapefile = gpd.read_file(filepath)
    data = shapefile.loc[:, shapefile.columns.isin(cols)]

    city_data_list.append(data)
    
    majority, maj_count, perc_list, city_perc = city_stats(data)
    city_dict = {'majority': majority, 
                  'maj_count': maj_count, 
                  'perc_list': perc_list, 
                  'city_perc': city_perc}
    
    city_dict_list.append(city_dict)
print(city_data_list[10])

            Settlement   kyc_pop  HISTO_0  HISTO_1  HISTO_2  HISTO_3  HISTO_4  \
0           Commercial    5782.0      1.0      0.0      0.0      0.0      0.0   
1     Kaberera Village    2465.0      0.0      0.0      0.0      1.0      0.0   
2             Kamwanya   12000.0      6.0      0.0      1.0      6.0      1.0   
3      Kandutu Village    1200.0      2.0      1.0      1.0      7.0      2.0   
4              Kuwinda    7000.0      3.0      0.0      0.0      3.0      1.0   
5             Kibagare   30000.0     25.0      0.0      2.0     17.0      0.0   
6   Githongori Village    6000.0     33.0      6.0     12.0     44.0     17.0   
7          Fuata Nyayo   22388.0      0.0      0.0      0.0      2.0      0.0   
8            Mariguini   30708.0      6.0      0.0      0.0      4.0      0.0   
9           Laini Saba  102240.0     32.0      0.0      1.0     10.0      0.0   
10          Kambi Muru    3591.0     29.0      0.0      0.0     10.0      0.0   
11              Kianda   600

In [21]:
# 0. accra
# 1. athiriver
# 2. capetown
# 3 daressalaam
# 4. kampala
# 5. kisumu
# 6. lagos
# 7. malabon
# 8. portharcourt
# 9. nairobi east
# 10. nairobi center

for city in city_dict_list:
    maj_indices = [index for (index, item) in enumerate(list(city.values())[0]) if item == 'HISTO_4']
    # print(maj_indices)

    # accrahisto3 = [city_data_list[0].loc[i] for i in maj_indices]
    # accrahisto3

# Parses through dictionary list and grabs all indices with HISTO_4 as majority
k=9
maj_indices = [index for (index, item) in enumerate(list(city_dict_list[k].values())[0]) if item == 'HISTO_0']
print(maj_indices)
# For grabbing population
accrahisto3 = [city_data_list[k].loc[i] for i in maj_indices]
popsum = 0
for city in accrahisto3:
    popsum = popsum + int(city['kyc_pop'])
print(popsum)

print(list(city_dict_list[k].values())[3])

[25, 29, 47]
26878
HISTO_0    15.015542
HISTO_1     2.295794
HISTO_2     4.372221
HISTO_3    11.754578
HISTO_4    65.794885
HISTO_5     0.766981
dtype: float64
